In [10]:
import copy, numpy as np
np.random.seed(0)

In [11]:
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [12]:
# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

In [13]:
# We're going to create a lookup table that maps from an integer to its binary representation. 
# The binary representations will be our input and output data for each math problem we try to get
# the network to solve. This lookup table will be very helpful in converting from integers to bit 
# strings.

# training dataset generation
int2binary = {}

# maximum length of the binary numbers we'll be adding.
binary_dim = 8

In [14]:
# This computes the largest number that is possible to represent with the binary 
# length we chose
largest_number = pow(2,binary_dim)

# This is a lookup table that maps from an integer to its binary representation.
binary = np.unpackbits(np.array([range(largest_number)],dtype=np.uint8).T,axis=1)

for i in range(largest_number):
    int2binary[i] = binary[i]

In [15]:
# input variables

#  learning rate
alpha = 0.1

# We are adding two numbers together, so we'll be feeding in two-bit strings one character at the 
# time each. Thus, we need to have two inputs to the network (one for each of the numbers being added).
input_dim = 2

# This is the size of the hidden layer that will be storing our carry bit.
hidden_dim = 16

# we're only predicting the sum, which is one number. Thus, we only need one output
output_dim = 1

In [7]:
# initialize neural network weights

# This is the matrix of weights that connects our input layer and our hidden layer.
# Thus, it has "input_dim" rows and "hidden_dim" columns. (2 x 16 unless you change it).
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1

# This is the matrix of weights that connects the hidden layer to the output layer.
# Thus, it has "hidden_dim" rows and "output_dim" columns. (16 x 1 unless you change it).
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) - 1

# This is the matrix of weights that connects the hidden layer in the previous time-step
# to the hidden layer in the current timestep. It also connects the hidden layer in the
# current timestep to the hidden layer in the next timestep (we keep using it). 
# Thus, it has the dimensionality of "hidden_dim" rows and "hidden_dim" columns. 
# (16 x 16 unless you change it).
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) - 1


# These store the weight updates that we would like to make for each of the weight matrices.
# After we've accumulated several weight updates, we'll actually update the matrices.
synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [16]:
# training logic
for j in range(10000):
    
    # generate a simple addition problem (a + b = c)
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding

    # true answer
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    # where we'll store our best guess (binary encoded)
    d = np.zeros_like(c)

    overallError = 0
    
    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))
    
    # moving along the positions in the binary encoding
    for position in range(binary_dim):
        
        # generate input and output
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))

        # output layer (new binary representation)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))

        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
    
        # decode estimate so we can print it out
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # store hidden layer so we can use it in the next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
    
    future_layer_1_delta = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        
        X = np.array([[a[position],b[position]]])
        layer_1 = layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]
        
        # error at output layer
        layer_2_delta = layer_2_deltas[-position-1]
        # error at hidden layer
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) +
                         layer_2_delta.dot(synapse_1.T)) *
        sigmoid_output_to_derivative(layer_1)

        # let's update all our weights so we can try again
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    

    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha    

    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    # print out progress
    if(j % 1000 == 0):
        print("Error:",overallError)
        print("Pred:",d)
        print("True:",c)
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")

Error: [ 0.21803299]
Pred: [0 1 0 1 1 0 1 1]
True: [0 1 0 1 1 0 1 1]
44 + 47 = 91
------------
Error: [ 0.14609197]
Pred: [0 1 1 0 1 1 1 0]
True: [0 1 1 0 1 1 1 0]
81 + 29 = 110
------------
Error: [ 0.1599709]
Pred: [0 1 1 0 0 1 1 0]
True: [0 1 1 0 0 1 1 0]
59 + 43 = 102
------------
Error: [ 0.1042304]
Pred: [0 1 1 1 0 1 0 1]
True: [0 1 1 1 0 1 0 1]
33 + 84 = 117
------------
Error: [ 0.20615389]
Pred: [1 0 1 1 0 0 0 1]
True: [1 0 1 1 0 0 0 1]
116 + 61 = 177
------------
Error: [ 0.16295477]
Pred: [0 1 1 1 0 1 0 0]
True: [0 1 1 1 0 1 0 0]
107 + 9 = 116
------------
Error: [ 0.098017]
Pred: [0 1 1 1 0 0 1 1]
True: [0 1 1 1 0 0 1 1]
0 + 115 = 115
------------
Error: [ 0.08861635]
Pred: [0 1 0 1 1 1 0 1]
True: [0 1 0 1 1 1 0 1]
25 + 68 = 93
------------
Error: [ 0.17292424]
Pred: [0 1 1 0 1 0 0 1]
True: [0 1 1 0 1 0 0 1]
79 + 26 = 105
------------
Error: [ 0.20454111]
Pred: [0 1 1 1 0 1 0 1]
True: [0 1 1 1 0 1 0 1]
75 + 42 = 117
------------
